In [1]:
import re
import collections
from utils.myutils import load_object, split_triple, save_object
from operator import itemgetter

In [3]:
file_path_owl = "dbpedia/dbpedia_2016-10.owl"
file_path_triples_filtered_sorted_by_relation = "dbpedia/mappingbased_literals_en_filtered_sorted_by_relation.ttl"

owl_file = open(file_path_owl, 'r', encoding='utf-8')
triples_file_filtered_sorted_by_relation = open(file_path_triples_filtered_sorted_by_relation, 'r', encoding='utf-8')

relations_valid = load_object('relations_valid')
instance_types_dict = load_object('instance_types_dict')

domain_dict = dict()

In [4]:
regex_xml_tag_subclass = re.compile('<rdfs:subClassOf rdf:resource=[^>]*/>')
regex_resource = re.compile('rdf:resource=".*"/>')

def find_most_special_class(classes):
    sub_dict = dict()
    super_dict = dict()
    for this_class in classes:
        search_term = 'rdf:about="' + this_class[1:-1] + '"'
        owl_file.seek(0)
        for line in owl_file:
            if search_term in line:
                #Read next line
                subclass_line = owl_file.readline()

                try:
                    rdfs_subclass_tag = regex_xml_tag_subclass.search(subclass_line)[0]
                except TypeError:
                    subclass_line = owl_file.readline()
                    try:
                        rdfs_subclass_tag = regex_xml_tag_subclass.search(subclass_line)[0]
                        print("WARNING! Could not find in-line subclassOf for class " + this_class)
                        print("Found subclassOf in the next line. Check whether this is correct!")
                    except TypeError:
                        print("No Superclass for " + this_class)
                        continue

                try:
                    regex_xml_tag_subclass.search(subclass_line)[1]
                except IndexError:
                    class_superclass = "<" + regex_resource.search(rdfs_subclass_tag)[0][14:-3] + ">"
                    print("Superclass for " + this_class + " found: " + class_superclass)

                    sub_dict[class_superclass] = this_class
                    super_dict[this_class] = class_superclass

                    continue
                print("ERROR! Multiple Findings! Second Finding: " + regex_xml_tag_subclass.search(subclass_line)[1])
    
    #Try to find any that has no SubClass -> most special ones
    most_special_classes = list()
    for this_class in classes:
        try:
            print("Look up " + this_class + " in dict")
            sub_dict[this_class]
        except KeyError:
            print("Most special Class: " + this_class)
            most_special_classes.append(this_class)
    
    #This should not occur - we would have a circular reference then
    if len(most_special_classes) == 0:
        print("ERROR! No special class!")
        return None
    
    if len(most_special_classes) == 1:
        print("Final most special class: " + most_special_classes[0])
        return most_special_classes[0][1:-1]
    else:
        print("Found more than one most special classes - Take that with more hierarchy")
        #Find the one that has more superclasses in the hierarchy
        most_special_classes_and_supers = list()
        for msc in most_special_classes:
            cnt = 0
            curr = msc
            while True:
                try:
                    curr = super_dict[curr]
                    cnt += 1
                except KeyError:
                    break
            most_special_classes_and_supers.append((msc,cnt))
        
        #TODO: Absteigend
        most_special_classes_and_supers.sort(key=itemgetter(1), reverse=True)
        print("Hierarchy-depth: " + str(most_special_classes_and_supers))
        
        return most_special_classes_and_supers[0][0][1:-1]

In [5]:
def infer_domain(relation):
    
    at_relation = False
    used_types = list()
    used_types_union = set()
    used_types_counter = collections.Counter()
    valid_triple_cnt = 0
    triples_file_filtered_sorted_by_relation.seek(0)
    for line in triples_file_filtered_sorted_by_relation:
        triple = split_triple(line)
    
        if triple[1][1:-1] != relation:
            if at_relation:
                break
            else:
                continue
        
        valid_triple_cnt += 1
        
        if not at_relation:
            at_relation = True
            used_types = instance_types_dict[triple[0]]
            for t in instance_types_dict[triple[0]]:
                used_types_counter[t] += 1
        else:
            try:
                #Intersection
                used_types = list(set(used_types) & set(instance_types_dict[triple[0]]))
                #Union
                used_types_union.update(instance_types_dict[triple[0]])
                for t in instance_types_dict[triple[0]]:
                    used_types_counter[t] += 1
            except KeyError:
                #Addition to strip
                lookup = triple[0]
                while True:              
                    matches = re.findall('<.+(__.+)>', lookup)
                    try:
                        lookup = lookup.replace(matches[0],'')
                    except IndexError:
                        break
                try:
                    used_types = list(set(used_types) & set(instance_types_dict[triple[0]]))
                    used_types_union.update(instance_types_dict[triple[0]])
                    for t in instance_types_dict[triple[0]]:
                        used_types_counter[t] += 1
                except KeyError:
                    print("WARNING! Remvoing additions does not give us insights! Ignoring " + str(lookup))
                    valid_triple_cnt -= 1
                    continue
    
    #Filter out non-DBPedia Types
    used_types = [ut for ut in used_types if 'dbpedia.org' in ut]
    used_types_union = [ut for ut in used_types_union if 'dbpedia.org' in ut]
    
    if len(used_types) == 1:
        print("Single Type found: " + used_types[0])
        return used_types[0][1:-1]
    
    else:
        print("No common type in relation " + relation + " or more than one common type")
        
        print("Union: " + str(used_types_union))
        
        parts_of = [(t[0], t[1] / valid_triple_cnt) for t in used_types_counter.most_common() if 'dbpedia.org' in t[0]]
        print("Parts of: " + str(parts_of))
        
        start = True
        classes = list()
        for part in parts_of:
            if start:
                start = False
                number = part[1]
            elif number - 0.01 > part[1]:
                break
            
            classes.append(part[0])
        
        print("Trying to find out most special class out of " + str(classes))
        
        return find_most_special_class(classes)        
        

In [6]:
regex_xml_tag = re.compile('<rdfs:domain rdf:resource=[^>]*/>')
regex_resource = re.compile('rdf:resource=".*"/>')

def get_domain(relation):
    #print(relation)
    search_term = 'rdf:about="' + relation + '"'
    owl_file.seek(0)
    for line in owl_file:
        if search_term in line:
            #Read next line
            domain_line = owl_file.readline()
            #print(str(relation) + " found in line " + line)
            #print("Assuming domain in line " + range_line)
            
            try:
                rdfs_domain_tag = regex_xml_tag.search(domain_line)[0]
            except TypeError:
                domain_line = owl_file.readline()
                try:
                    rdfs_domain_tag = regex_xml_tag.search(domain_line)[0]
                    print("WARNING! Could not find in-line domain for relation " + relation)
                    print("Found domain in the next line. Check whether this is correct!")
                except TypeError:
                    print("Try to infer a domain for relation " + relation)
                    return infer_domain(relation)
            
            try:
                regex_xml_tag.search(domain_line)[1]
            except IndexError:
                relation_domain = regex_resource.search(rdfs_domain_tag)[0][14:-3]
                
                return relation_domain
            print("ERROR! Multiple Findings! Second Finding: " + regex_xml_tag.search(domain_line)[1])
    
    return None

In [7]:
cnt = 0
for relation in relations_valid:
    domain = get_domain(relation)
    
    
    
    if domain is None:
        cnt += 1
        print("ERROR! Domain of " + relation + " is None!")
    else:
        domain_dict[relation] = domain
        print(relation + ": " + str(domain))
    
    

print(cnt)
owl_file.close()

http://dbpedia.org/ontology/ranking: http://dbpedia.org/ontology/School
http://dbpedia.org/ontology/shareDate: http://dbpedia.org/ontology/Broadcaster
http://dbpedia.org/ontology/shipLaunch: http://dbpedia.org/ontology/Ship
http://dbpedia.org/ontology/introductionDate: http://dbpedia.org/ontology/MeanOfTransportation
http://dbpedia.org/ontology/cylinderBore: http://dbpedia.org/ontology/AutomobileEngine
http://dbpedia.org/ontology/co2Emission: http://dbpedia.org/ontology/AutomobileEngine
Try to infer a domain for relation http://dbpedia.org/ontology/dateOfAbandonment
No common type in relation http://dbpedia.org/ontology/dateOfAbandonment or more than one common type
Union: ['<http://dbpedia.org/ontology/Infrastructure>', '<http://dbpedia.org/ontology/Airport>', '<http://dbpedia.org/ontology/RouteOfTransportation>', '<http://dbpedia.org/ontology/Road>', '<http://dbpedia.org/ontology/ArchitecturalStructure>', '<http://dbpedia.org/ontology/Location>', '<http://dbpedia.org/ontology/Place>'

No common type in relation http://dbpedia.org/ontology/productionStartYear or more than one common type
Union: ['<http://dbpedia.org/ontology/Engine>', '<http://dbpedia.org/ontology/AutomobileEngine>', '<http://dbpedia.org/ontology/Device>', '<http://dbpedia.org/ontology/MeanOfTransportation>', '<http://dbpedia.org/ontology/Aircraft>', '<http://dbpedia.org/ontology/Automobile>']
Parts of: [('<http://dbpedia.org/ontology/MeanOfTransportation>', 0.9538019232675721), ('<http://dbpedia.org/ontology/Automobile>', 0.7978586299196986), ('<http://dbpedia.org/ontology/Aircraft>', 0.1559432933478735), ('<http://dbpedia.org/ontology/Engine>', 0.0465946267472985), ('<http://dbpedia.org/ontology/Device>', 0.0465946267472985), ('<http://dbpedia.org/ontology/AutomobileEngine>', 0.0465946267472985)]
Trying to find out most special class out of ['<http://dbpedia.org/ontology/MeanOfTransportation>']
Superclass for <http://dbpedia.org/ontology/MeanOfTransportation> found: <http://www.w3.org/2002/07/owl#T

No common type in relation http://dbpedia.org/ontology/orbits or more than one common type
Union: ['<http://dbpedia.org/ontology/MeanOfTransportation>', '<http://dbpedia.org/ontology/SpaceShuttle>']
Parts of: [('<http://dbpedia.org/ontology/MeanOfTransportation>', 1.0), ('<http://dbpedia.org/ontology/SpaceShuttle>', 1.0)]
Trying to find out most special class out of ['<http://dbpedia.org/ontology/MeanOfTransportation>', '<http://dbpedia.org/ontology/SpaceShuttle>']
Superclass for <http://dbpedia.org/ontology/MeanOfTransportation> found: <http://www.w3.org/2002/07/owl#Thing>
Superclass for <http://dbpedia.org/ontology/SpaceShuttle> found: <http://dbpedia.org/ontology/MeanOfTransportation>
Look up <http://dbpedia.org/ontology/MeanOfTransportation> in dict
Look up <http://dbpedia.org/ontology/SpaceShuttle> in dict
Most special Class: <http://dbpedia.org/ontology/SpaceShuttle>
Final most special class: <http://dbpedia.org/ontology/SpaceShuttle>
http://dbpedia.org/ontology/orbits: http://db

No common type in relation http://dbpedia.org/ontology/activeYearsStartDate or more than one common type
Union: ['<http://dbpedia.org/ontology/PersonFunction>', '<http://dbpedia.org/ontology/MeanOfTransportation>', '<http://dbpedia.org/ontology/Congressman>', '<http://dbpedia.org/ontology/GridironFootballPlayer>', '<http://dbpedia.org/ontology/Athlete>', '<http://dbpedia.org/ontology/Politician>', '<http://dbpedia.org/ontology/Governor>', '<http://dbpedia.org/ontology/Economist>', '<http://dbpedia.org/ontology/Work>', '<http://dbpedia.org/ontology/Person>', '<http://dbpedia.org/ontology/Sound>', '<http://dbpedia.org/ontology/Ambassador>', '<http://dbpedia.org/ontology/PrimeMinister>', '<http://dbpedia.org/ontology/Cricketer>', '<http://dbpedia.org/ontology/Noble>', '<http://dbpedia.org/ontology/TennisPlayer>', '<http://dbpedia.org/ontology/Criminal>', '<http://dbpedia.org/ontology/Document>', '<http://dbpedia.org/ontology/Senator>', '<http://dbpedia.org/ontology/MemberOfParliament>', '

No common type in relation http://dbpedia.org/ontology/budget or more than one common type
Union: ['<http://dbpedia.org/ontology/Film>', '<http://dbpedia.org/ontology/TennisTournament>', '<http://dbpedia.org/ontology/ArchitecturalStructure>', '<http://dbpedia.org/ontology/Wikidata:Q11424>', '<http://dbpedia.org/ontology/SocietalEvent>', '<http://dbpedia.org/ontology/Album>', '<http://dbpedia.org/ontology/Organisation>', '<http://dbpedia.org/ontology/Library>', '<http://dbpedia.org/ontology/Tournament>', '<http://dbpedia.org/ontology/SportsEvent>', '<http://dbpedia.org/ontology/Work>', '<http://dbpedia.org/ontology/MusicalWork>', '<http://dbpedia.org/ontology/Building>', '<http://dbpedia.org/ontology/School>', '<http://dbpedia.org/ontology/Event>', '<http://dbpedia.org/ontology/Place>', '<http://dbpedia.org/ontology/Agent>', '<http://dbpedia.org/ontology/GovernmentAgency>', '<http://dbpedia.org/ontology/Location>', '<http://dbpedia.org/ontology/EducationalInstitution>']
Parts of: [('<ht

...

No common type in relation http://dbpedia.org/ontology/areaMetro or more than one common type
Union: ['<http://dbpedia.org/ontology/Country>', '<http://dbpedia.org/ontology/Settlement>', '<http://dbpedia.org/ontology/Region>', '<http://dbpedia.org/ontology/AdministrativeRegion>', '<http://dbpedia.org/ontology/Place>', '<http://dbpedia.org/ontology/Town>', '<http://dbpedia.org/ontology/Village>', '<http://dbpedia.org/ontology/City>', '<http://dbpedia.org/ontology/PopulatedPlace>', '<http://dbpedia.org/ontology/Location>']
Parts of: [('<http://dbpedia.org/ontology/PopulatedPlace>', 1.0), ('<http://dbpedia.org/ontology/Place>', 1.0), ('<http://dbpedia.org/ontology/Location>', 1.0), ('<http://dbpedia.org/ontology/Settlement>', 0.9845956354300385), ('<http://dbpedia.org/ontology/City>', 0.6225930680359435), ('<http://dbpedia.org/ontology/Town>', 0.04621309370988447), ('<http://dbpedia.org/ontology/Country>', 0.010269576379974325), ('<http://dbpedia.org/ontology/Village>', 0.0064184852374839

No common type in relation http://dbpedia.org/ontology/activeYearsEndYear or more than one common type
Union: ['<http://dbpedia.org/ontology/FormulaOneRacer>', '<http://dbpedia.org/ontology/Poet>', '<http://dbpedia.org/ontology/RacingDriver>', '<http://dbpedia.org/ontology/Politician>', '<http://dbpedia.org/ontology/Governor>', '<http://dbpedia.org/ontology/ArtistDiscography>', '<http://dbpedia.org/ontology/Guitarist>', '<http://dbpedia.org/ontology/BasketballPlayer>', '<http://dbpedia.org/ontology/SumoWrestler>', '<http://dbpedia.org/ontology/Work>', '<http://dbpedia.org/ontology/MusicalWork>', '<http://dbpedia.org/ontology/Scientist>', '<http://dbpedia.org/ontology/Medician>', '<http://dbpedia.org/ontology/Cleric>', '<http://dbpedia.org/ontology/Wrestler>', '<http://dbpedia.org/ontology/Cardinal>', '<http://dbpedia.org/ontology/FashionDesigner>', '<http://dbpedia.org/ontology/Location>', '<http://dbpedia.org/ontology/AmericanFootballPlayer>', '<http://dbpedia.org/ontology/SnookerWorl

No common type in relation http://dbpedia.org/ontology/sourceConfluenceElevation or more than one common type
Union: ['<http://dbpedia.org/ontology/BodyOfWater>', '<http://dbpedia.org/ontology/River>', '<http://dbpedia.org/ontology/Stream>', '<http://dbpedia.org/ontology/Location>', '<http://dbpedia.org/ontology/Place>', '<http://dbpedia.org/ontology/NaturalPlace>']
Parts of: [('<http://dbpedia.org/ontology/Place>', 1.0), ('<http://dbpedia.org/ontology/Location>', 1.0), ('<http://dbpedia.org/ontology/Stream>', 0.9951923076923077), ('<http://dbpedia.org/ontology/BodyOfWater>', 0.9951923076923077), ('<http://dbpedia.org/ontology/NaturalPlace>', 0.9951923076923077), ('<http://dbpedia.org/ontology/River>', 0.9951923076923077)]
Trying to find out most special class out of ['<http://dbpedia.org/ontology/Place>', '<http://dbpedia.org/ontology/Location>', '<http://dbpedia.org/ontology/Stream>', '<http://dbpedia.org/ontology/BodyOfWater>', '<http://dbpedia.org/ontology/NaturalPlace>', '<http://

No common type in relation http://dbpedia.org/ontology/electionMajority or more than one common type
Union: ['<http://dbpedia.org/ontology/Congressman>', '<http://dbpedia.org/ontology/Politician>', '<http://dbpedia.org/ontology/Governor>', '<http://dbpedia.org/ontology/Person>', '<http://dbpedia.org/ontology/PrimeMinister>', '<http://dbpedia.org/ontology/MemberOfParliament>', '<http://dbpedia.org/ontology/Chancellor>', '<http://dbpedia.org/ontology/Agent>', '<http://dbpedia.org/ontology/Mayor>', '<http://dbpedia.org/ontology/OfficeHolder>', '<http://dbpedia.org/ontology/Judge>', '<http://dbpedia.org/ontology/President>']
Parts of: [('<http://dbpedia.org/ontology/Person>', 1.0), ('<http://dbpedia.org/ontology/Agent>', 1.0), ('<http://dbpedia.org/ontology/Politician>', 0.5751824817518248), ('<http://dbpedia.org/ontology/OfficeHolder>', 0.4233576642335766), ('<http://dbpedia.org/ontology/MemberOfParliament>', 0.37761557177615573), ('<http://dbpedia.org/ontology/Governor>', 0.0233576642335

No common type in relation http://dbpedia.org/ontology/width or more than one common type
Union: ['<http://dbpedia.org/ontology/MountainRange>', '<http://dbpedia.org/ontology/RouteOfTransportation>', '<http://dbpedia.org/ontology/AutomobileEngine>', '<http://dbpedia.org/ontology/River>', '<http://dbpedia.org/ontology/MeanOfTransportation>', '<http://dbpedia.org/ontology/ArchitecturalStructure>', '<http://dbpedia.org/ontology/Settlement>', '<http://dbpedia.org/ontology/ProtectedArea>', '<http://dbpedia.org/ontology/Lake>', '<http://dbpedia.org/ontology/RoadTunnel>', '<http://dbpedia.org/ontology/Automobile>', '<http://dbpedia.org/ontology/Bridge>', '<http://dbpedia.org/ontology/Region>', '<http://dbpedia.org/ontology/AdministrativeRegion>', '<http://dbpedia.org/ontology/Locomotive>', '<http://dbpedia.org/ontology/NaturalPlace>', '<http://dbpedia.org/ontology/Building>', '<http://dbpedia.org/ontology/SpaceStation>', '<http://dbpedia.org/ontology/Tunnel>', '<http://dbpedia.org/ontology/We

In [8]:
save_object(domain_dict, "domain_dict")